In [172]:
import pandas as pd
import os
# import linecache
import datetime
from datetime import datetime as dt
import numpy as np

In [196]:
def payer_category(payer):
    payers = {
        "Deidentified Minimum": 'min',
        'Deidentified Maximum': 'max'
    }

    try:
        return payers[payer]
    except KeyError:
        return 'payer'

In [211]:
file = 'Standard Charges_AnMed Cannon_2023.xlsx'
tin = '57-0342027'
ccn = '420011'
actual_file = "Absolute Total Care Medicaid.csv"

In [199]:
# date_updated = linecache.getline(actual_file, 2).split(": ")[-1].split(",")[0]
# date_updated = dt.strptime(date_updated, '%m/%d/%Y').strftime('%Y-%m-%d')

date_updated = '2023-01-01'

In [175]:
df = pd.read_csv('.\\input_files\\' + actual_file, skiprows=4)

In [176]:
df.rename(columns = {
    'ContractName': 'payer_name',
    'CodeType': 'code_type',
    'Code': 'code',
    'Description': 'description',
    'Service Description': 'rev_desc',
    'Base Rate Methodology': 'rev_method',
    'ServiceCode': 'patient_class',
    'Negotiated Charge / Base Rate': 'rate'
}, inplace = True)

In [177]:
# df['payer_orig'] = df['payer_name']
df['code_orig'] = df['code']

In [178]:
df.columns

Index(['payer_name', 'code_type', 'code', 'description', 'rev_desc',
       'rev_method', 'rate', 'patient_class', 'Deidentified Minimum',
       'Deidentified Maximum', 'code_orig'],
      dtype='object')

In [179]:
df['code_type'] = df['code_type'].str.lower()

In [180]:
df['patient_class'] = df['patient_class'].str.replace('IP', 'inpatient').str.replace('OP', 'outpatient')
# Set rev_code equal to code and set code_type to None if code_type equals revenuecode
df.loc[df["code_type"] == "revenuecode", "rev_code"] = df["code"]

In [181]:
df.loc[df["code_type"] == "revenuecode"]

,payer_name,code_type,code,description,rev_desc,rev_method,rate,patient_class,Deidentified Minimum,Deidentified Maximum,code_orig,rev_code


In [183]:
df1 = df.drop(['payer_name', 'rate'], axis=1)
cols = df1.columns.tolist()
value_vars = ['Deidentified Minimum', 'Deidentified Maximum']
id_vars = [x for x in cols if x not in value_vars]

df1 = pd.melt(df1, id_vars=id_vars, value_vars=value_vars, var_name='payer_name', value_name='rate')


In [184]:
df.drop(value_vars, axis=1, inplace=True)

In [185]:
df_cols = df.columns.values.tolist()
df_cols.sort()

df  = df[df_cols]
df1 = df1[df_cols]

In [186]:
df3 = df.append(df1)
df = df3

C:\Users\adria\AppData\Local\Temp\ipykernel_23360\658878721.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df.append(df1)


In [187]:
df['rate'] = df['rate'].str.replace('$', '').str.replace(',', '')

C:\Users\adria\AppData\Local\Temp\ipykernel_23360\505826267.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['rate'] = df['rate'].str.replace('$', '').str.replace(',', '')


In [207]:
df['payer_orig'] = df['payer_name']
df.loc[df["code_type"] == "revenuecode", ["code", "code_type"]] = [None, None]

# Set code_prefix based on code_type value using numpy.where
df["code_prefix"] = np.where(df["code_type"] == "cpt", "hcpcs_cpt",
                             np.where(df["code_type"] == "drg", "apr-drg", None))

In [201]:
# Add constants
df['hospital_tin'] = tin
df['hospital_ccn'] = ccn
df['filename'] = file
df['url'] = 'https://anmed.org/sites/default/files/2023-01/Standard%20Charges_AnMed%20Cannon_2023.xlsx'

df['rev_code'] = df['rev_code'].fillna('na')
df['payer_category'] = df['payer_name'].apply(payer_category)
df['file_last_updated'] = date_updated

In [213]:
df.to_csv("./output_files/" + actual_file, index=False)

In [209]:
df

,code,code_orig,code_type,description,patient_class,payer_name,rate,rev_code,rev_desc,rev_method,payer_orig,code_prefix,hospital_tin,hospital_ccn,filename,url,payer_category,file_last_updated
0,0011,0011,drg,LIVER TRANSPLANT &/OR INTESTINAL TRANSPLANT,inpatient,Absolute Total Care Medicaid,71954.45,na,Medical/Surgical,Relative Weights,Absolute Total Care Medicaid,apr-drg,57-0342027,420011,Standard Charges_AnMed Cannon_2023.xlsx,https://anmed.org/sites/default/files/2023-01/...,payer,2023-01-01
1,0012,0012,drg,LIVER TRANSPLANT &/OR INTESTINAL TRANSPLANT,inpatient,Absolute Total Care Medicaid,72356.34,na,Medical/Surgical,Relative Weights,Absolute Total Care Medicaid,apr-drg,57-0342027,420011,Standard Charges_AnMed Cannon_2023.xlsx,https://anmed.org/sites/default/files/2023-01/...,payer,2023-01-01
2,0013,0013,drg,LIVER TRANSPLANT &/OR INTESTINAL TRANSPLANT,inpatient,Absolute Total Care Medicaid,84988.92,na,Medical/Surgical,Relative Weights,Absolute Total Care Medicaid,apr-drg,57-0342027,420011,Standard Charges_AnMed Cannon_2023.xlsx,https://anmed.org/sites/default/files/2023-01/...,payer,2023-01-01
3,0014,0014,drg,LIVER TRANSPLANT &/OR INTESTINAL TRANSPLANT,inpatient,Absolute Total Care Medicaid,147802.81,na,Medical/Surgical,Relative Weights,Absolute Total Care Medicaid,apr-drg,57-0342027,420011,Standard Charges_AnMed Cannon_2023.xlsx,https://anmed.org/sites/default/files/2023-01/...,payer,2023-01-01
4,0021,0021,drg,HEART &/OR LUNG TRANSPLANT,inpatient,Absolute Total Care Medicaid,78456.79,na,Medical/Surgical,Relative Weights,Absolute Total Care Medicaid,apr-drg,57-0342027,420011,Standard Charges_AnMed Cannon_2023.xlsx,https://anmed.org/sites/default/files/2023-01/...,payer,2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17873,T1001,T1001,cpt,Nursing assessment/evaluatn,outpatient,Deidentified Maximum,184.80,na,OPHOSP,Per Unit,Deidentified Maximum,hcpcs_cpt,57-0342027,420011,Standard Charges_AnMed Cannon_2023.xlsx,https://anmed.org/sites/default/files/2023-01/...,max,2023-01-01
17874,U0001,U0001,cpt,2019-ncov diagnostic p,outpatient,Deidentified Maximum,45.62,na,OPHOSP,Per Unit,Deidentified Maximum,hcpcs_cpt,57-0342027,420011,Standard Charges_AnMed Cannon_2023.xlsx,https://anmed.org/sites/default/files/2023-01/...,max,2023-01-01
17875,U0002,U0002,cpt,Covid-19 lab test non-cdc,outpatient,Deidentified Maximum,65.18,na,OPHOSP,Per Unit,Deidentified Maximum,hcpcs_cpt,57-0342027,420011,Standard Charges_AnMed Cannon_2023.xlsx,https://anmed.org/sites/default/files/2023-01/...,max,2023-01-01
17876,U0003,U0003,cpt,Cov-19 amp prb hgh thruput,outpatient,Deidentified Maximum,127.03,na,OPHOSP,Per Unit,Deidentified Maximum,hcpcs_cpt,57-0342027,420011,Standard Charges_AnMed Cannon_2023.xlsx,https://anmed.org/sites/default/files/2023-01/...,max,2023-01-01
